# Segregation calculation

In [28]:
import numpy as np
import pandas as pd
from decimal import Decimal
import time

In [29]:
# Import python script with segreg functions
from segregationMetrics import Metrics

In [30]:
cc = Metrics()

In [31]:
'''
Data prepared as id,  x, y , sum, attribute 1, attributes 2, attributes 3, Attribute n
'''

# cc.readAttributesFile('res/ks201_oa_ethnic.csv')
cc.readAttributesFile('data/AP2010_CEM_RMSP_EDU.csv')

matrix([[  2.00000000e+00,   3.67940727e+05,   7.41220127e+06, ...,
           7.21700000e+03,   3.22600000e+03,   3.57000000e+02],
        [  1.00000000e+00,   3.65279005e+05,   7.41308577e+06, ...,
           3.94100000e+03,   8.16000000e+02,   1.87000000e+02],
        [  3.00000000e+00,   3.62939248e+05,   7.41398731e+06, ...,
           6.76300000e+03,   1.40000000e+03,   3.12000000e+02],
        ..., 
        [  6.09000000e+02,   3.25291157e+05,   7.36486407e+06, ...,
           5.57900000e+03,   1.09400000e+03,   7.64000000e+02],
        [  6.25000000e+02,   3.18200304e+05,   7.38684501e+06, ...,
           6.72800000e+03,   2.58500000e+03,   8.80000000e+01],
        [  6.29000000e+02,   3.16921299e+05,   7.38553520e+06, ...,
           7.60000000e+03,   1.03900000e+03,   2.96000000e+02]])

**Compute population intensity**

Distance matrix is calculated at this step

In [32]:
'''
Parameters:
bandwidth - is set to be 5000m by default, you can change it here
weightmethod - 1 for gaussian, 2 for bi-square and empty for moving window
'''

start_time = time.time()
cc.locality = cc.cal_localityMatrix(bandwidth=3000, weightmethod=1)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.2933969497680664 seconds ---


In [33]:
# List of coordinates from file (x, y) for validation
cc.location

matrix([[  367940.72713,  7412201.27442],
        [  365279.00478,  7413085.76939],
        [  362939.24837,  7413987.31233],
        ..., 
        [  325291.15667,  7364864.06947],
        [  318200.30414,  7386845.00788],
        [  316921.29889,  7385535.20008]])

In [34]:
# Display local population intensity for groups
cc.locality

array([[  8279.8462652 ,   3600.89814857,   5388.00321297,   1689.44639886,
           268.56572077],
       [  8511.69276191,   3566.9300309 ,   5274.27540802,   1377.54685989,
           262.73938253],
       [  9577.65965392,   3886.71905402,   5551.25579861,   1203.69229422,
           277.13216749],
       ..., 
       [ 14997.89485778,   5860.56765295,   6171.27284685,   1068.69496683,
           605.38805821],
       [ 11150.70559268,   5233.81760202,   7196.39321957,   2867.14714079,
           304.8247775 ],
       [ 11290.66084299,   5200.98310512,   6712.36558737,   2011.08553609,
           304.41121067]])

**Compute local dissimilarity**

In [35]:
# Function to compute local dissimilarity
d_localsimilarity = cc.cal_localDissimilarity()
# To select a specific element or range, use the [] with the index value
d_localsimilarity[0: 10]

array([  9.55860422e-05,   6.61707339e-05,   1.59987254e-04,
         1.26048662e-04,   2.87121813e-04,   1.41013920e-04,
         1.40268287e-04,   1.57788605e-04,   4.80449181e-04,
         2.88238193e-04])

In [36]:
# Fution to compute global dissimilarity
d_globaldis = cc.cal_globalDissimilarity()
# Display gloabl value
d_globaldis

0.12053398250135673

In [37]:
''''
rs is a matrix of n_group * n_group
therefore, exposure (m,n) = rs[m,n]
m,m = isolation index of group m
m,n = expouse index of group m to n
'''
expo = cc.cal_globalExposure()

In [38]:
# Result of all combinations of groups expousure
expo

array([[ 0.41024676,  0.1941277 ,  0.27317189,  0.11046947,  0.01198418],
       [ 0.40289845,  0.19287   ,  0.27481578,  0.11760663,  0.01180913],
       [ 0.38953139,  0.18881944,  0.27740802,  0.13282396,  0.0114172 ],
       [ 0.32715575,  0.16767302,  0.27551352,  0.21947063,  0.01018708],
       [ 0.40602988,  0.19276895,  0.27157291,  0.11685554,  0.01277272]])

In [39]:
# To select a specific expousure of m to n, use the []
expo[0]

array([ 0.41024676,  0.1941277 ,  0.27317189,  0.11046947,  0.01198418])

In [40]:
expo_local = cc.cal_localExposure()

ValueError: setting an array element with a sequence.

In [27]:
expo_local

NameError: name 'expo_local' is not defined

In [18]:
d_localsimilarity = np.asmatrix(d_localsimilarity).transpose()

In [20]:
# Displays the dissimilaty indexes, use [] to select a value or range
# d_localsimilarity[0:10]

**Prepare data for saving on a local file**

In [20]:
rs1 = np.concatenate((cc.locality, d_localsimilarity), axis=1)

In [21]:
N_attri = cc.locality.shape[1]

names = ['MSOA_ID','x','y','sum']

for i in range(N_attri):
    names.append('group_'+str(i))
    
for i in range(N_attri):
    names.append('locality_'+str(i))
    
names.append('local_dissimilarity')

In [22]:
#sf = shapefile.Reader('res/resolution_msoa_2011.shp')
#shp = sf.records()
#shp = np.asmatrix(shp)
#rs = np.concatenate((shp[:,0], rs), axis=1)

rs = np.concatenate((cc.attriMatrix,rs1),axis = 1)

** Save results to a local file**

The first paramenter ("res/rs1.csv") is the path/filename or use only filename if results are being saved on the same folder. The name should be changed for new executions or the result will be 

In [23]:
# Change the name of file for results saving
# It's recomended to use the name of the input file adding the sufix ..._result
rs = pd.DataFrame(rs, columns=names);
rs.to_csv("result/rs1.csv", sep=",", index = False)